* To understand more in details [refer](./tool.md)

# ⚒️ Tools in LangChain

## Create tools using `@tool` decorator

In [ ]:
from langchain_core.tools import tool


@tool("multiply_two_numbers")
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

# OR

# For async


@tool
def amultiply(a: int, b: int) -> int:
    """multiply two number"""
    return a * b


# Without providing "multiply_two_number" @tool default set function name as a tool name.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply_two_numbers
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


### Using [`Annotated`](../additionals/annotated.ipynb) in Tool

In [9]:
from typing import Annotated, List


@tool
def multiply_max(a: Annotated[int, "scale factor"], b: Annotated[List[int], "list of ints over which to take maximum"]) -> int:
    """Multiply a by the max of b"""
    return a * max(b)


print(multiply_max.args_schema.model_json_schema())

{'description': 'Multiply a by the max of b', 'properties': {'a': {'description': 'scale factor', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'list of ints over which to take maximum', 'items': {'type': 'integer'}, 'title': 'B', 'type': 'array'}}, 'required': ['a', 'b'], 'title': 'multiply_max', 'type': 'object'}


### Also we can customize JSON schema

In [10]:
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


@tool("multiplication-tool", args_schema=CalculatorInput, return_direct=True)
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

multiplication-tool
Multiply two numbers.
{'a': {'description': 'first number', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number', 'title': 'B', 'type': 'integer'}}
True


### Tools are also support docstring like Google Style docstring and langChain convert into JSON schema

In [15]:
@tool(parse_docstring=True)
def foo(bar: str, baz: int) -> str:
    """The foo.

    Args:
        bar: The bar.
        baz: The baz.
    """
    return bar


print(foo.args_schema.model_json_schema())

{'description': 'The foo.', 'properties': {'bar': {'description': 'The bar.', 'title': 'Bar', 'type': 'string'}, 'baz': {'description': 'The baz.', 'title': 'Baz', 'type': 'integer'}}, 'required': ['bar', 'baz'], 'title': 'foo', 'type': 'object'}


### Build tools using StructureTool

In [8]:
from langchain_core.tools import StructuredTool


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(func=multiply, coroutine=amultiply)

print(calculator.invoke({"a": 2, "b": 3}))
print(await calculator.ainvoke({"a": 2, "b": 5}))

6
10


In [ ]:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool


class CalculateInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(
    func=multiply,
    name="calculator",
    description="multiply two numbers",
    args_schema=CalculateInput,
    return_direct=True,
    # coroutine= ... <- you can specify an async method if desired as well
)

print(calculator.invoke({"a": 2, "b": 3}))
print(calculator.name)
print(calculator.description)
print(calculator.args)

6
calculator
multiply two numbers
{'a': {'description': 'first number', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number', 'title': 'B', 'type': 'integer'}}
{'properties': {'a': {'description': 'first number', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number', 'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'CalculateInput', 'type': 'object'}


## Creating Tools using [runnables](../additionals/runnable.md)

## Creating Tools using BaseTool

In [ ]:
from typing import Optional
from langchain_core.callbacks import (
    AsyncCallbackManagerForToolRun, CallbackManagerForToolRun)
from langchain_core.tools import BaseTool
from langchain_core.tools.base import ArgsSchema
from pydantic import Field, BaseModel


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    a: int = Field(description="second number")

# Note: It's important that every field has type hints. BaseTool is a Pydantic class and not having type hints can lead unexpected behavior.


class CustomCalculatorTool(BaseTool):
    name: str = "calculator"
    description: str = "useful for when you need to solve math problems."
    args_schema: Optional[ArgsSchema] = CalculatorInput
    return_direct: bool = True

    def _run(self, a: int, b: int, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Use the tool"""
        return a * b

    async def _arun(self, a: int, b: int, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously"""

        return self._run(a, b, run_manager=run_manager.get_sync())


multiply = CustomCalculatorTool()
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

print(multiply.invoke({"a": 2, "b": 3}))
print(await multiply.ainvoke({"a": 2, "b": 3}))

calculator
useful for when you need to solve math problems.
{'a': {'description': 'second number', 'title': 'A', 'type': 'integer'}}
True


TypeError: CustomCalculatorTool._run() missing 1 required positional argument: 'b'

### How to create async tools

### How to handling tool errors

In [1]:
from langchain_core.tools import ToolException, StructuredTool


def get_weather(city: str):
    """
    Get the weather of give city name


    Args:
        city: city name
    """
    raise ToolException(f"Error: There is no city by the name of {city}")


get_weather_tool = StructuredTool.from_function(
    func=get_weather,
    handle_tool_error=True,
)

get_weather_tool.invoke({"city": "foo"})

'Error: There is no city by the name of foo'

* We can define string instead of `handle_tool_error = True`, `handle_tool_error="string"`, In here when any exception occur return this string any error.

In [17]:
get_weather_tool = StructuredTool.from_function(
    func=get_weather,
    handle_tool_error="There is no such city, but it's probably above 0K there!"
)

get_weather_tool.invoke({"city": "foo"})

"There is no such city, but it's probably above 0K there!"

- We can define **function** also.

In [ ]:
def _handle_error(error: ToolException) -> str:
    return f"The following errors occurred during tool execution: `{error.args[0]}`"


get_weather_tool = StructuredTool.from_function(
    func=get_weather,
    handle_tool_error=_handle_error
)


get_weather_tool.invoke({"city": "foobar"})

'The following errors occurred during tool execution: `Error: There is no city by the name of foobar`'

In [10]:
from langchain_core.tools import tool, ToolException, StructuredTool


def get_weather(location: str) -> str:
    """Give the weather of given city name"""
    if location.lower() == "unknown":
        raise ToolException(f"Error: Unknown location '{location}'.")
    return f"The weather in {location} is sunny."


get_weather_tool = StructuredTool.from_function(
    func=get_weather,
    handle_tool_error=True
)

get_weather_tool.invoke({"location": "unknown"})

"Error: Unknown location 'unknown'."